<a href="https://colab.research.google.com/github/rayn180/gomoku/blob/master/YourTTS_zeroshot_VC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo zero-shot Voice conversion with YourTTS

##TTS Model setup

### Download and install Coqui TTS


In [1]:
!git clone https://github.com/Edresson/Coqui-TTS -b multilingual-torchaudio-SE TTS
!pip install -q -e TTS/
!pip install -q torchaudio==0.9.0

Cloning into 'TTS'...
remote: Enumerating objects: 25680, done.
remote: Total 25680 (delta 0), reused 0 (delta 0), pack-reused 25680 (from 1)
Receiving objects: 100% (25680/25680), 133.71 MiB | 26.07 MiB/s, done.
Resolving deltas: 100% (18791/18791), done.
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Could not find a version that satisfies the requirement torchaudio==0.9.0 (from versions: 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2

###Download TTS Checkpoint

In [2]:
# TTS checkpoints

# download config
!gdown --id 1-PfXD66l1ZpsZmJiC-vhL055CDSugLyP
# download language json
! gdown --id 1_Vb2_XHqcC0OcvRF82F883MTxfTRmerg
# download speakers json
! gdown --id 1SZ9GE0CBM-xGstiXH2-O2QWdmSXsBKdC -O speakers.json
# download checkpoint
! gdown --id 1sgEjHt0lbPSEw9-FSbC_mBoOPwNi87YR -O best_model.pth.tar

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1-PfXD66l1ZpsZmJiC-vhL055CDSugLyP

but Gdown can't. Please check connections and permissions.
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with

### Imports

In [15]:
import sys
TTS_PATH = "TTS/"

# add libraries into environment
sys.path.append(TTS_PATH) # set this if TTS is not installed globally

import os
import string
import time
import argparse
import json

import numpy as np
import IPython
from IPython.display import Audio


import torch

from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
try:
  from TTS.utils.audio import AudioProcessor
except:
  from TTS.utils.audio import AudioProcessor


from TTS.tts.models import setup_model
from TTS.config import load_config
from TTS.tts.models.vits import *

RuntimeError: 
----------------------------------------------------------

Failed initializing MeCab. Please see the README for possible solutions:

    https://github.com/SamuraiT/mecab-python3#common-issues

If you are still having trouble, please file an issue here, and include the
ERROR DETAILS below:

    https://github.com/SamuraiT/mecab-python3/issues

issueを英語で書く必要はありません。

------------------- ERROR DETAILS ------------------------
arguments: 
default dictionary path: None
[pos != std::string::npos] format error: /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic
----------------------------------------------------------


In [14]:
# Japonca MeCab destek dosyalarını yükle
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8

# Python bağlantısını kur
!pip install mecab-python3

# mecabrc dosyasının yolunu manuel olarak belirle
!echo `mecab-config --dicdir`"/mecab-ipadic" > /usr/local/etc/mecabrc


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libmecab-dev is already the newest version (0.996-14build9).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-3).
mecab is already the newest version (0.996-14build9).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [12]:
# Sistem kütüphanelerini yükle
!apt-get install mecab libmecab-dev mecab-ipadic-utf8 -y

# Python binding'ini yükle
!pip install mecab-python3


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 7,367 kB of archives.
After this operation, 59.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab2 amd64 0.996-14build9 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libmecab-dev amd64 0.996-14build9 [306 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-utils amd64 0.996-14build9 [4,850 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 mecab-ipadic all 2.7.0-20070801+main-3 [6,718 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mecab amd64 0.996-14build9 [136 kB]
Get:6 http://archive.ubuntu.co

In [10]:
!pip install pypinyin


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.0/837.0 kB 19.6 MB/s eta 0:00:00


In [8]:
!pip install anyascii


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.4 MB/s eta 0:00:00


In [6]:
!pip install gruut


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 109.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
  Created wheel for gruut: filename=gruut-2.4.0-py3-none-any.whl size=86758 sha256=308ace4f16adb46a96a80abe22e4d275850b9e1d57c43ba9edb4d1377a54f756
  Stored in directory: /root/.cache/pip/wheels/7f/ff/c1/291e91d9b767694fb7f30f6cd16a5b89d6d67afacd796ad842
  Created wheel for gruut-ipa: filename=gruut_ipa-0.13.0-py3-none-any.whl size=104873 sha256=5e8e667

### Paths definition

In [ ]:
OUT_PATH = 'out/'

# create output path
os.makedirs(OUT_PATH, exist_ok=True)

# model vars
MODEL_PATH = 'best_model.pth.tar'
CONFIG_PATH = 'config.json'
TTS_LANGUAGES = "language_ids.json"
TTS_SPEAKERS = "speakers.json"
USE_CUDA = torch.cuda.is_available()

### Restore model

In [ ]:
# load the config
C = load_config(CONFIG_PATH)


# load the audio processor
ap = AudioProcessor(**C.audio)

speaker_embedding = None

C.model_args['d_vector_file'] = TTS_SPEAKERS
C.model_args['use_speaker_encoder_as_loss'] = False

model = setup_model(C)
model.language_manager.set_language_ids_from_file(TTS_LANGUAGES)
# print(model.language_manager.num_languages, model.embedded_language_dim)
# print(model.emb_l)
cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
# remove speaker encoder
model_weights = cp['model'].copy()
for key in list(model_weights.keys()):
  if "speaker_encoder" in key:
    del model_weights[key]

model.load_state_dict(model_weights)


model.eval()

if USE_CUDA:
    model = model.cuda()

# synthesize voice
use_griffin_lim = False

##Speaker encoder setup

### Install helper libraries

In [ ]:
! pip install -q pydub ffmpeg-normalize==1.21.0

### Paths definition

In [ ]:
CONFIG_SE_PATH = "config_se.json"
CHECKPOINT_SE_PATH = "SE_checkpoint.pth.tar"

# download config
! gdown --id  19cDrhZZ0PfKf2Zhr_ebB-QASRw844Tn1 -O $CONFIG_SE_PATH
# download checkpoint
! gdown --id   17JsW6h6TIh7-LkU2EvB_gnNrPcdBxt7X -O $CHECKPOINT_SE_PATH

###Imports

In [ ]:
from TTS.tts.utils.speakers import SpeakerManager
from pydub import AudioSegment
from google.colab import files
import librosa

###Load the Speaker encoder

In [ ]:
SE_speaker_manager = SpeakerManager(encoder_model_path=CHECKPOINT_SE_PATH, encoder_config_path=CONFIG_SE_PATH, use_cuda=USE_CUDA)

###Define helper function

In [ ]:
def compute_spec(ref_file):
  y, sr = librosa.load(ref_file, sr=ap.sample_rate)
  spec = ap.spectrogram(y)
  spec = torch.FloatTensor(spec).unsqueeze(0)
  return spec

##Voice conversion

###Upload, normalize and resample your wav files

Please upload wav files

In [ ]:
print("Select target speaker reference audios files:")
target_files = files.upload()
target_files = list(target_files.keys())
for sample in target_files:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f

In [ ]:
print("Select driving speaker reference audios files:")
driving_files = files.upload()
driving_files = list(driving_files.keys())
for sample in driving_files:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f

In [ ]:
print("Select driving audio file:")
driving_file = files.upload()
driving_file = list(driving_file.keys())
for sample in driving_file:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f
driving_file = driving_file[0]

###Compute embeddings

In [ ]:
target_emb = SE_speaker_manager.compute_d_vector_from_clip(target_files)
target_emb = torch.FloatTensor(target_emb).unsqueeze(0)

In [ ]:
driving_emb = SE_speaker_manager.compute_d_vector_from_clip(driving_files)
driving_emb = torch.FloatTensor(driving_emb).unsqueeze(0)

###Convert the voice

In [ ]:
driving_spec = compute_spec(driving_file)
y_lengths = torch.tensor([driving_spec.size(-1)])
if USE_CUDA:
    ref_wav_voc, _, _ = model.voice_conversion(driving_spec.cuda(), y_lengths.cuda(), driving_emb.cuda(), target_emb.cuda())
    ref_wav_voc = ref_wav_voc.squeeze().cpu().detach().numpy()
else:
    ref_wav_voc, _, _ = model.voice_conversion(driving_spec, y_lengths, driving_emb, target_emb)
    ref_wav_voc = ref_wav_voc.squeeze().detach().numpy()


print("Reference Audio after decoder:")
IPython.display.display(Audio(ref_wav_voc, rate=ap.sample_rate))

In [ ]:
!pip install -q gruut